### Import Libraries

In [4]:
try:
    from elasticsearch import Elasticsearch,helpers
    import os
    import sys
    import elasticsearch
    import pandas as pd
except Exception as e:
    print("The following error occured:",e)

### Connect to Elasticsearch

In [16]:
#Lets connect to the server, we are using the python client to do so

def connect_database():
    es = None
    es = Elasticsearch([{'host':'localhost','port':9200}])
    if es.ping():
        print("connected")
    else:
        print("Failed")
    return es
connect_database()

connected


C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [25]:
#### Check files in directory

In [4]:
for filename in os.listdir():
    print(filename)

.git
.ipynb_checkpoints
combined_stock_data.csv
data
ElasticSearchbasics.ipynb
Elk_google_loc.ipynb
nlpextractor.ipynb
stock_project.ipynb
Untitled.ipynb


### Import File

In [6]:
df = pd.read_json(r"C:\Users\parth\2022\notebooks\data\Takeout\Location History\Semantic Location History\2021\2021_DECEMBER.json")
# print(df.isnull().sum())

In [7]:
df[:8]

,timelineObjects
0,{'activitySegment': {'startLocation': {'latitu...
1,{'placeVisit': {'location': {'latitudeE7': 406...
2,{'activitySegment': {'startLocation': {'latitu...
3,{'placeVisit': {'location': {'latitudeE7': 406...
4,{'placeVisit': {'location': {'latitudeE7': 406...
5,{'placeVisit': {'location': {'latitudeE7': 406...
6,{'placeVisit': {'location': {'latitudeE7': 406...
7,{'activitySegment': {'startLocation': {'latitu...


### Convert file to right format
Right format should have the following attributes
<li>Dict format</li>
<li>Unique ID</li>

In [9]:
df2 = df.to_dict('records')

In [10]:
# add ids to the documents
for i in range(len(df2)):
    df2[i]["id"] = i

In [11]:
# check what the data looks like now
for c,line in enumerate(df2):
    print(c,line)
    break

0 {'timelineObjects': {'activitySegment': {'startLocation': {'latitudeE7': 406270797, 'longitudeE7': -740243470, 'sourceInfo': {'deviceTag': 657537}}, 'endLocation': {'latitudeE7': 406254813, 'longitudeE7': -740240442, 'sourceInfo': {'deviceTag': 657537}}, 'duration': {'startTimestamp': '2021-12-01T19:55:01.136Z', 'endTimestamp': '2021-12-01T20:04:06.189Z'}, 'distance': 201, 'activityType': 'WALKING', 'confidence': 'HIGH', 'activities': [{'activityType': 'WALKING', 'probability': 90.53097367286682}, {'activityType': 'IN_PASSENGER_VEHICLE', 'probability': 4.127069190144539}, {'activityType': 'CYCLING', 'probability': 2.461812272667885}, {'activityType': 'IN_BUS', 'probability': 0.8563749492168421}, {'activityType': 'STILL', 'probability': 0.828239694237709}, {'activityType': 'RUNNING', 'probability': 0.47490238212049}, {'activityType': 'IN_SUBWAY', 'probability': 0.22579405922442602}, {'activityType': 'IN_TRAM', 'probability': 0.18923425814136802}, {'activityType': 'MOTORCYCLING', 'prob

In [20]:
def generator(df2,database_name,table_name,id_column):
    for c,line in enumerate(df2):
        if(line.get("timelineObjects").get("placeVisit")!=None):
            print(c)

            yield{
                '_index' : database_name,
                '_type' : table_name,
                '_id': line.get(id_column),
                '_source': {

                        'location_latitude':line.get("timelineObjects").get("placeVisit").get("location").get("latitudeE7"),
                        'location_longitude':line.get("timelineObjects").get("placeVisit").get("location").get("longitudeE7"),
                        'address':line.get("timelineObjects").get("placeVisit").get("location").get("address"),
                        'location_name':line.get("timelineObjects").get("placeVisit").get("location").get("name"),
                        'start_timestamp':line.get("timelineObjects").get("placeVisit").get("duration").get("startTimestamp"),
                        'end_timestamp':line.get("timelineObjects").get("placeVisit").get("duration").get("endTimestamp")
                        # 'distance_meters':line.get("timelineObjects",['No Data']).get("activitySegment",['No Data']).get("waypointPath",['No Data']).get("distanceMeters",['No Data']),
                        # 'travel_mode':line.get("timelineObjects",['No Data']).get("activitySegment",['No Data']).get("waypointPath",['No Data']).get("travelMode",['No Data'])                                                                                                
                    # get returns the value of a key

                }
            }
    # raise StopIteration
    

In [21]:
gen = generator(df2,"g_test","glocation","id")

In [22]:
next(gen)

1


{'_index': 'g_test',
 '_type': 'glocation',
 '_id': 1,
 '_source': {'location_latitude': 406254253,
  'location_longitude': -740240437,
  'address': '8007 5th Ave\nBrooklyn, NY 11209\nUSA',
  'location_name': 'Farmacon Pharmacy',
  'start_timestamp': '2021-12-01T20:04:06.189Z',
  'end_timestamp': '2021-12-01T20:10:13.193Z'}}

In [23]:
es.indices.create(index="g_test2",ignore = 400)

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [g_test2/aO_nIgMrRzO8mx09rkaqfQ] already exists',
    'index_uuid': 'aO_nIgMrRzO8mx09rkaqfQ',
    'index': 'g_test2'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [g_test2/aO_nIgMrRzO8mx09rkaqfQ] already exists',
  'index_uuid': 'aO_nIgMrRzO8mx09rkaqfQ',
  'index': 'g_test2'},
 'status': 400}

In [24]:
try:
    res = helpers.bulk(es,generator(df2,"g_test2","glocation","id"))
    print("Working")
except Exception as e:
    print("done",e)


1
3
4
5
6
8
11
13
15
17
19
23
25
27
29
31
33
35
37
39
41
44
48
50
52
55
57
59
61
63
65
67
70
72
76
78
80
82
84
86
88
90
93
95
97
99
101
103
106
108
Working


C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [13]:
# Database setup

In [14]:
es.indices.create(index = "mydatabasename",ignore = 400)

C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'mydatabasename'}

In [123]:
res = es.indices.get_alias("*")
for name in res:
    print(name)

.kibana-event-log-7.16.3-000003
g_test
icecream_kaggle
.async-search
.kibana_7.16.3_001
greencan_demo
.kibana-event-log-7.16.3-000001
.kibana_task_manager_7.16.3_001
.apm-custom-link
.tasks
.apm-agent-configuration
stocks2020to2021
netflix_sample
.kibana-event-log-7.16.3-000002
person


C:\Users\parth\AppData\Local\Temp\ipykernel_31560\516184771.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.indices.get_alias("*")
C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: this request accesses system indices: [.async-search, .kibana_7.16.3_001, .kibana_task_manager_7.16.3_001, .apm-custom-link, .tasks, .apm-agent-configuration], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchWarning)


In [16]:
es.indices.delete(index='mydatabasename',ignore = [400,404])

C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True}

## Inserting data into kibana

In [19]:
record1 = {
    "name" : "Parth",
    "age" :26,
    
    
}
record2 = {
    "name" : "Samarth",
    "age" :20,
    
    
}

In [20]:
es.indices.create(index="person",ignore = 400)

C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'person'}

In [17]:
resp = es.index(index = "g_test",doc_type='glocation',body = df2[0])

C:\Users\parth\AppData\Local\Temp\ipykernel_5688\2801456637.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  resp = es.index(index = "g_test",doc_type='glocation',body = df2[0])
C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


In [19]:
for i in range(len(df)):
    es.index(index = "g_test2",doc_type='glocation',body = df2[i])
    

C:\Users\parth\AppData\Local\Temp\ipykernel_5688\2628710241.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index = "g_test2",doc_type='glocation',body = df2[i])


In [18]:
resp

{'_index': 'g_test',
 '_type': 'glocation',
 '_id': '8gPnbIAB-WKrEUGjvlT_',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 2}

In [141]:
query = {
    "query" : {
        "match_all":{}
    }
}
    
res = es.search(index='g_test',body = query   )

C:\Users\parth\AppData\Local\Temp\ipykernel_31560\1110477524.py:7: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index='g_test',body = query   )
